In [1]:
import os

In [2]:
import numpy as np

In [3]:
import pandas as pd

In [80]:
from collections import defaultdict
import tarfile
import gc
import warnings
import six
import argparse
from PIL import Image

In [81]:
input_dir = '/home/fenoms/meta_data/miniImagenet/csv'

data_dir = '/home/fenoms/meta_data/miniImagenet/'

In [82]:
def _read_image_as_array(path, dtype='int32'):
    f = Image.open(path)
    
    try:
        image = np.asarray(f, dtype=dtype)
        
    finally:
        if hasattr(f, 'close'):
            f.close()
    return image


In [88]:
def download_imgs():
    train = pd.read_csv(input_dir + '/train.csv', sep=',')
    test = pd.read_csv(input_dir + '/test.csv', sep=',')
    val = pd.read_csv(input_dir + '/val.csv', sep=',')
    
    labels = train.label.unique().tolist() + test.label.unique().tolist() + val.label.unique().tolist()

    for label in labels:
        os.system('wget "http://image-net.org/download/synset?wnid=' + label + '&username=fenoms&accesskey=acdcf71fbeafbdbc2cb0036e89b1c614346c733e&release=latest&src=stanford" -O /home/fenoms/meta_data/miniImagenet/' + label + '.tar')

In [ ]:
download_imgs()

In [76]:
def resize_save(data_path, out_file):
	dataset = pd.read_csv(data_path, sep=',')
	lbls = dataset.label.unique().tolist()
	tmp_dict = defaultdict(list)
	file_names = []
	lbl_names = []
	for lbl in lbls:
		print lbl
		tar = tarfile.open(data_dir + lbl + ".tar")
		imgs = tar.getmembers()
		np.random.shuffle(imgs)
		# imgs = imgs[:600]
		c = 0
		for img in imgs:
			f = tar.extractfile(img)
			try:
				img_array = _read_image_as_array(f)
				img_array = scipy.misc.imresize(img_array, (84, 84))
				img_array = img_array.astype('float32')
				img_array *= (1.0 / 255.0)
				tmp_dict[lbl].append(img_array.reshape((1, 84, 84, 3)))
				file_names.append(img.name)
				lbl_names.append(lbl)
				c += 1
			# Reading/resizing some image throws exception, so had to filter them out. 
			except Exception, e:
				print "skipping image..."
			if c == 600:
				break
	results = {key : np.concatenate(value) for key, value in tmp_dict.items()}
	np.savez(data_dir + out_file + ".npz", **results)
	sub = pd.DataFrame({'img_file': file_names, 'label': lbl_names})
	sub.to_csv(data_dir + out_file + '.csv', index=False)

In [77]:

print "train..."
resize_save(input_dir + '/train.csv', 'train')
print "test..."
resize_save(input_dir + '/test.csv', 'test')
print "dev..."
resize_save(input_dir + '/val.csv', 'dev')


train...
n01532829


ReadError: file could not be opened successfully